In [3]:
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
data = pd.read_csv('C:/Users/USER/Desktop/MOHIM/PFA/ScrapeStorm/Complet/HardDataSet.csv',low_memory=False,encoding='latin-1')

In [5]:
#print(data.columns.tolist())

In [6]:
list_companies = data['Company_Name']
list_jobs = data['Title']
list_skills = data['Skills']

In [7]:
#len(list_jobs)

In [8]:
for country in list_skills:
    print(country)

web_development;MVC;GIT;testing;Gulp;Webpack;JIRA;English;communication
VueJS;HTML;CSS;Javascript;standards W3C;responsive_design;PHP;LESS;SASS;D3;Leaflet;Highcharts;Photoshop;Illustrator;Git
Javascript;ES 2015;Typescript;NodeJS;Electron;C#;software_architecture;React;Redux;WebPack
HTML;CSS;AWS;English;French
iOS;Android;PHP;git;Wordspress;Drupal;Prestashop
HTML;CSS;web_design;IT;English;French;Spanish;German
Symfony2;MySQL;PHP;HTML;CSS;jQuery;Javascript
Java;PHP;NodeJS;SQL Server;MySQL;MongoDB;PostgreSQL;HTML;CSS;Javascript;AngularJS;Spring;Hibernate;Maven;SVN;Highchart;Python;Django
DOM;Javascript;TypeScript;CSS;AngularJS;React
Javascript;CSS;HTML;NodeJS;React
Oracle_Database;PL/SQL;SQL;HTML;CSS;Javascript;REST Web;Oracle;Python;R;Perl;Ruby;Git
UX;UI;Agile;PHP;jQuery;Wordpress;Magento;SQL
CMS;Git;CSS;SASS;jQuery;Javascript;HTML;Agile;Scrum
RubyOnRails;React;Android;Ruby;Javascript;Java;C++;DevOps;Docker;Swarm;Laravel;R&D;Django
MySQL;PHP;Symfony;Bootstrap;JQuery;VueJS;React;Docker;Ap

In [9]:
def get_unique_jobs(list_jobs):
    unique_jobs = []
    for job in list_jobs:
        if(job not in unique_jobs):
            unique_jobs.append(job)
    return unique_jobs


In [10]:
def get_score_job(job,vector_user_job,vectorizer):
    vector_unique_job = vectorizer.transform(job)
    list_similarity = linear_kernel(vector_user_job,vector_unique_job).flatten()
    score_similarity = sum(list_similarity)/len(list_similarity)
    return score_similarity,job
    

In [11]:
def get_similar_jobs(user_job,unique_jobs):

    vectorizer = TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)
    vectorizer.fit(unique_jobs)
    list_general = []
    #list_general = [[]]
    #list_general.append([])
    vector_user_job = vectorizer.transform(user_job)
    for job in unique_jobs:
        job = [job]
      
        list_general.append(get_score_job(job,vector_user_job,vectorizer))
    return list_general

In [12]:
unique_jobs = []
unique_jobs = get_unique_jobs(list_jobs)

In [13]:
for job in unique_jobs:
    print(job)

web developer
Data scientist
Security engineer
Software engineer
Software engineer in Test
BI consultant


In [14]:
def get_skills_of_specific_job(given_job,list_skills,list_jobs):
    list_skills_of_given_job = []
    key_element = ""
    for i in range(0,len(list_jobs)):
        if list_jobs[i] == given_job:
            list_cleaned = list_skills[i].replace(";"," ")
            list_skills_of_given_job.append(list_cleaned)
            key_element = key_element + list_cleaned
    return key_element
  
           
    

In [15]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

In [16]:
def extract_topn_from_vector(feature_names, sorted_items, topn):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx] 
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [17]:
def recommand_skills(my_list,topN):
    #generate tf-idf for the given document
    vectorizer = TfidfVectorizer(use_idf=True, smooth_idf=True)
    my_list =[my_list]
    cv = vectorizer.fit(my_list)
    feature_names = cv.get_feature_names()
    tf_idf_vector = []
   #apply tf-idf vectorizer on all the skills
    for i in range(len(my_list)):
        text = [my_list[i]]
        tf_idf_vector = vectorizer.transform(text)
        #gather all the vectors in one vector
        if i == 0 :
            final_element = tf_idf_vector
        else :
            final_element = final_element + tf_idf_vector
    #sort the tf-idf vectors by descending order of scores
    sorted_items = sort_coo(final_element.tocoo())
    #extract the top n keywords
    recommended_skills = extract_topn_from_vector(feature_names,sorted_items,topN)
    return recommended_skills

In [18]:
#user_job = "data administrator"
print("enter the title of a specific job please")
user_job=input()
user_job = [user_job]
general = get_similar_jobs(user_job,unique_jobs)

general = sorted(general, reverse = True)

#print(general)
if  general[0][0] > 0.70:
    given_job = general[0][1][0]
    list_skills_of_given_job = get_skills_of_specific_job(given_job,list_skills,list_jobs)
    #list_skills_of_given_job
    final_list = recommand_skills(list_skills_of_given_job,20)
    print("the most required skills for this job on the market:")
    for element in final_list:
        print(element)
    #print(final_list)


else:
    print("Sadly we dont have enough infromations about the required job !")



enter the title of a specific job please
security engineer
the most required skills for this job on the market:
python
java
javascript
french
azure
aws
ruby
linux
ios
english
docker
cissp
ci
ceh
cd
xp
wifi
scrum
scala
oracle_database
